In [41]:
import pandas as pd
import pymorphy2 
import json, re

In [42]:
df_attributes = pd.read_parquet('../data/test/attributes_test.parquet', columns=["variantid", "characteristic_attributes_mapping"], engine='pyarrow')


In [43]:
df_attributes.head()

,variantid,characteristic_attributes_mapping
0,47920382,"{""Цвет товара"": [""бежевый"", ""светло-розовый""],..."
1,49801845,"{""Количество в упаковке, шт"": [""1""], ""Бренд"": ..."
2,49853444,"{""Бренд"": [""Vervaco""], ""Тип"": [""Набор для выши..."
3,49893028,"{""Цвет товара"": [""серый""], ""Ширина, см"": [""0.8..."
4,49987483,"{""Цвет товара"": [""разноцветный""], ""Название цв..."


In [44]:
def is_number(s):
    try:
        float(s)  # Попробуем преобразовать строку в float
        return True
    except ValueError:
        return False

def get_uniq_attributes(df):
    """
    Функция возвращает dict из уникальных атрибутов

    return: dict

    """
    result_dict = dict()
    for row in df[:]['characteristic_attributes_mapping']:
        #print(row)
        row_dct = json.loads(row)


        for key, val in row_dct.items():
            if result_dict.get(key) is None:
                result_dict[key] = (is_number(val[0]), val)
            elif result_dict.get(key)[0]:
                result_dict[key] = (is_number(val[0]), val)
                
    return result_dict


In [45]:
uniq_attr = get_uniq_attributes(df_attributes)

In [46]:
uniq_attr

{'Цвет товара': (False, ['бежевый', 'светло-розовый']),
 'Пол ребенка': (False, ['Унисекс']),
 'Бренд': (False, ['Funny Ducks']),
 'Тип': (False, ['Игрушка для ванной']),
 'Страна-изготовитель': (False, ['Китай']),
 'Развитие навыков': (False, ['Воображение', 'Кругозор', 'Моторика']),
 'Код ТРУ': (False, ['']),
 'Минимальный возраст ребенка': (False, ['От 3 лет']),
 'Материал': (False, ['ПВХ пластизоль']),
 'Вес товара, г': (False, ['подвесная люстра (лампа не в комплекте)']),
 'Максимальный возраст ребенка': (False, ['До 18 лет']),
 'Количество в упаковке, шт': (False, ['with a charging case']),
 'Материалы набора': (False, ['Акрил', 'Бумага', 'Канва', 'Металл', 'Хлопок']),
 'Техника вышивки': (False, ['Простой крест']),
 'Возрастные ограничения': (False, ['От 7 лет']),
 'Длина по горизонтали, см': (False, ['200 мм']),
 'Длина по вертикали, см': (False, ['200 мм']),
 'Размеры, мм': (False, ['400 х 400']),
 'Ширина, см': (False, ['29 см']),
 'Состав ниток': (False, ['Ткань']),
 'Длина,

In [47]:
num_attr = list(filter(lambda x: x[1][0], zip(uniq_attr.keys(),uniq_attr.values())))
num_attr

[('Максимальная продолжительность партии, мин.', (True, ['30'])),
 ('Плотность наполнителя, г/м2', (True, ['300'])),
 ('Количество предметов в комплекте', (True, ['1'])),
 ('Расход, кг/м2', (True, ['0.35'])),
 ('Рекомендуемое количество слоев', (True, ['2'])),
 ('Срок годности в днях', (True, ['540'])),
 ('Длина шампура, см', (True, ['75'])),
 ('Количество шампуров, шт', (True, ['6'])),
 ('Толщина, мм', (True, ['0.06'])),
 ('Количество цветов, шт', (True, ['16'])),
 ('Внутренний диаметр брашинга, мм', (True, ['18'])),
 ('Оптическая сила', (True, ['+4.00'])),
 ('Радиус кривизны', (True, ['8.5'])),
 ('Влагосодержание, %', (True, ['55'])),
 ('Количество внешних карманов, шт', (True, ['1'])),
 ('Плотность ткани, г/кв.м', (True, ['1'])),
 ('Количество насадок', (True, ['3'])),
 ('Объем чаши, л', (True, ['0.4'])),
 ('Макс. мощность (при блокировке мотора), Вт', (True, ['2000'])),
 ('Производительность, кг/мин', (True, ['2.5'])),
 ('Количество предметов', (True, ['6'])),
 ('Макс. мощность СВЧ

In [58]:
import re
from natasha import (
    Doc,
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger
)

# Инициализация компонентов Natasha
segmenter = Segmenter()
morph_vocab = MorphVocab()
news_embeddings = NewsEmbedding()
morph_tagger = NewsMorphTagger(news_embeddings)

def preprocess_text(text):
    # Удаляем цифры и пунктуацию, приводим к нижнему регистру
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = text.strip().lower()

    # Создаем документ для обработки Natasha
    doc = Doc(text)
    doc.segment(segmenter)  # Разбиваем на токены
    doc.tag_morph(morph_tagger)  # Морфологический анализ

    # Лемматизация с помощью MorphVocab
    normalized_words = []
    for token in doc.tokens:
        token.lemmatize(morph_vocab)  # Лемматизация
        normalized_words.append(token.lemma)  # Добавляем лемму в список

    # Объединяем леммы обратно в строку
    processed_text = ' '.join(normalized_words)

    return processed_text

In [59]:
from sentence_transformers import SentenceTransformer, util
import torch
model = SentenceTransformer('sergeyzh/rubert-tiny-turbo')

sentences = list(map(lambda x: x[0], num_attr)) #num_attr #['товар цвет', ]
preproc_sentences = list(map(lambda x: preprocess_text(x), sentences))
embeddings = model.encode(preproc_sentences)
print(preproc_sentences)

['максимальный продолжительность партия мина', 'плотность наполнитель гм', 'количество предмет в комплект', 'расход кгм', 'рекомендовать количество слой', 'срок годность в день', 'длина шампур см', 'количество шампуров шт', 'толщина мм', 'количество цвет шт', 'внутренний диаметр брашинг мм', 'оптический сила', 'радиус кривизна', 'влагосодержание', 'количество внешний карман шт', 'плотность ткань гквма', 'количество насадка', 'объем чаша л', 'макс мощность при блокировка мотор вт', 'производительность кгмин', 'количество предмет', 'макс мощность свч вт', 'время мойка при обычный программа мина', 'вместимость комплект', 'количество программа', 'расход вода за цикл л', 'количество пружина на спальный место', 'макс вес на спальный место кг', 'содержание какао', 'макс световой поток люмен', 'колвый светодиод', 'макс время работа от батарея ч', 'дальность освещение м', 'колвый режим', 'число отделение шт', 'вес перчатка oz унция', 'диаметр динамика мм', 'вес с упаковка г', 'шаг зуб tpi', 'дл

In [60]:
embeddings[0].shape

(312,)

In [61]:
def embeddings_to_torch(embeddings):
    return [torch.from_numpy(emb).unsqueeze(0) for emb in embeddings]

In [72]:
def clastarize(sentences, torch_embeddings, threshold=0.9):
    distribution = dict()
    clast_lst = []
    already_in = []
    clast_index = -1
    for i in range(len(sentences)):
        if i in already_in:
            continue
        already_in.append(i)
        clast_index += 1
        clast = []
        distribution[sentences[i]] = clast_index
        clast.append((sentences[i], 1.0))
        emb_i = torch_embeddings[i]
        for j in range(i + 1, len(sentences)):
            if j in already_in:
                continue
            emb_j = torch_embeddings[j]
            cos_sim = torch.cosine_similarity(emb_i, emb_j)
            if cos_sim > threshold:
                clast.append((sentences[j], cos_sim))
                distribution[sentences[j]] = clast_index
                already_in.append(j)
        clast_lst.append(clast)
    return clast_lst, distribution


In [63]:
torch_embeddings = embeddings_to_torch(embeddings)
num_clast_lst, num_clast_distribution = clastarize(sentences, torch_embeddings)
print(len(num_clast_lst))

696


In [64]:
for clast in num_clast_lst:
    print(f"clast main: {clast[0][0]}")
    print(f"clasts: {clast[1:]}")
    print("\n")

clast main: Максимальная продолжительность партии, мин.
clasts: [('Длительность, мин', tensor([0.9105]))]


clast main: Плотность наполнителя, г/м2
clasts: [('Плотность, кг/м3', tensor([0.9092])), ('Плотность, г/см3', tensor([0.9178]))]


clast main: Количество предметов в комплекте
clasts: [('Количество предметов', tensor([0.9375])), ('Количество в комплекте, шт.', tensor([0.9615])), ('Кол-во в комплекте, шт', tensor([0.9152])), ('Количество инструментов в наборе, шт.', tensor([0.9225])), ('Количество модулей в комплекте', tensor([0.9189])), ('Число предметов, шт', tensor([0.9268])), ('Количество колонок в комплекте', tensor([0.9378])), ('Количество наволочек в комплекте', tensor([0.9521])), ('Количество предметов в наборе', tensor([0.9632])), ('Число предметов в наборе, шт', tensor([0.9515])), ('Число пейджеров в комплекте, шт.', tensor([0.9091])), ('Количество фишек в наборе', tensor([0.9011])), ('Общее количество пакетов, шт', tensor([0.9136]))]


clast main: Расход, кг/м2
clasts: 

In [65]:
num_clast_distribution["Длина шампура, см"]

6

Теперь то же самое но с числовыми

In [66]:
sent_attr = list(filter(lambda x: not x[1][0], zip(uniq_attr.keys(),uniq_attr.values())))
sent_attr

[('Цвет товара', (False, ['бежевый', 'светло-розовый'])),
 ('Пол ребенка', (False, ['Унисекс'])),
 ('Бренд', (False, ['Funny Ducks'])),
 ('Тип', (False, ['Игрушка для ванной'])),
 ('Страна-изготовитель', (False, ['Китай'])),
 ('Развитие навыков', (False, ['Воображение', 'Кругозор', 'Моторика'])),
 ('Код ТРУ', (False, [''])),
 ('Минимальный возраст ребенка', (False, ['От 3 лет'])),
 ('Материал', (False, ['ПВХ пластизоль'])),
 ('Вес товара, г', (False, ['подвесная люстра (лампа не в комплекте)'])),
 ('Максимальный возраст ребенка', (False, ['До 18 лет'])),
 ('Количество в упаковке, шт', (False, ['with a charging case'])),
 ('Материалы набора',
  (False, ['Акрил', 'Бумага', 'Канва', 'Металл', 'Хлопок'])),
 ('Техника вышивки', (False, ['Простой крест'])),
 ('Возрастные ограничения', (False, ['От 7 лет'])),
 ('Длина по горизонтали, см', (False, ['200 мм'])),
 ('Длина по вертикали, см', (False, ['200 мм'])),
 ('Размеры, мм', (False, ['400 х 400'])),
 ('Ширина, см', (False, ['29 см'])),
 ('Со

In [67]:
sentences = list(map(lambda x: x[0], sent_attr)) #num_attr #['товар цвет', ]
preproc_sentences = list(map(lambda x: preprocess_text(x), sentences))
embeddings = model.encode(preproc_sentences)
print(preproc_sentences)

['цвет товар', 'пол ребенок', 'бренд', 'тип', 'странаизготовитель', 'развитие навык', 'код тереть', 'минимальный возраст ребенок', 'материал', 'вес товар г', 'максимальный возраст ребенок', 'количество в упаковка шт', 'материал набор', 'техника вышивка', 'возрастной ограничение', 'длина по горизонталь см', 'длина по вертикаль см', 'размер мм', 'ширина см', 'состав нитка', 'длина м', 'название цвет', 'минимальный продолжительность партия мина', 'минимальный число игрок', 'максимальный число игрок', 'вид выпуск товар', 'целевой аудитория', 'вид настольный игра', 'целевой аудитория игра', 'артикул', 'вес кг', 'размерность', 'материал чехол', 'материал наполнитель', 'технология пошив', 'комплектация', 'объем мл', 'фильтр', 'тип цоколь', 'вид лампа', 'количество лампа шт', 'макс мощность лампа вт', 'материал плафон', 'вид питание', 'цвет плафон', 'площадь освещение квм', 'тип выключатель', 'партномер', 'состав', 'упаковка', 'назначение средство для стирка', 'название аромат', 'аромат', 'раз

In [73]:
torch_embeddings = embeddings_to_torch(embeddings)
sent_clast_lst, sent_clast_distribution = clastarize(sentences, torch_embeddings, 0.92)
print(len(sent_clast_lst))

2608


In [74]:
print(len(sentences))
for clast in sent_clast_lst:
    print(f"clast main: {clast[0][0]}")
    print(f"clasts: {clast[1:]}")
    print("\n")

4621
clast main: Цвет товара
clasts: []


clast main: Пол ребенка
clasts: []


clast main: Бренд
clasts: [('Страна бренда', tensor([0.9476])), ('Поддерживаемые бренды', tensor([0.9500])), ('Совместимый бренд', tensor([0.9412]))]


clast main: Тип
clasts: [('Формат', tensor([0.9209])), ('Тип застежки', tensor([0.9224])), ('Вид принта', tensor([0.9220])), ('Тип сушки', tensor([0.9217])), ('Вид', tensor([0.9456])), ('Тип застёжки', tensor([0.9224])), ('Форма', tensor([0.9276])), ('Тип основы', tensor([0.9535])), ('Тип часов', tensor([0.9229])), ('Тип пронации', tensor([0.9494])), ('Тип вставки', tensor([0.9371])), ('Тип установки', tensor([0.9309])), ('Тип конфорок', tensor([0.9353])), ('Тип духовки', tensor([0.9323])), ('Тип коннектора 1', tensor([0.9515])), ('Тип коннектора 2', tensor([0.9515])), ('Тип реза', tensor([0.9233])), ('Тип канвы', tensor([0.9298])), ('Тип соединения', tensor([0.9266])), ('Категория', tensor([0.9201])), ('Типоразмер', tensor([0.9401])), ('Тип расцепления', ten

In [ ]:
num_attr_length = len()